In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

<div dir="rtl">
    به دلیل اینکه در محیط Kaggle در حال توسعه هستم، شکل ورودی ها متفاوت هستند
</div>

In [ ]:
train_data_dir = '/kaggle/input/gender-classification-dataset/Training'
val_data_dir = '/kaggle/input/gender-classification-dataset/Validation'

In [ ]:
paths_dict = {
    'female': [],
    'male': []
}

In [ ]:
for key in paths_dict.keys():
    for dirname, _, filenames in os.walk(os.path.join(train_data_dir, key)):
        for filename in filenames:
            paths_dict[key].append(os.path.join(dirname, filename))

In [ ]:
groups = [key + '\n' + str(len(paths_dict[key])) 
          for key in paths_dict.keys()]
count_data = [len(paths_dict[key])
          for key in paths_dict.keys()]
colors = ['b', 'r']

Visualization of the amount of train data in classes

In [ ]:
plt.title('Amount of train data')

width = len(count_data) * 0.3
plt.bar(groups, count_data, width=width, color=colors, alpha=0.6, bottom=2, linewidth=2)

Creation of directories for generators

In [ ]:
train_dir = '/kaggle/working/train_dir'
test_dir = '/kaggle/working/test_dir'

In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    
    for key in paths_dict.keys():
        os.makedirs(os.path.join(dir_name, key))

In [ ]:
create_directory(train_dir)
create_directory(test_dir)

In [ ]:
def copy_images(start_index, end_index, paths, dest_dir):
    for i in range(start_index, end_index):
        dest_path = os.path.join(dest_dir, paths[i].split('/')[5])
        shutil.copy2(paths[i], dest_path)

In [ ]:
# Part of the test data set
test_data_proportion = 0.2

In [ ]:
for key in paths_dict.keys():
    test_index = len(paths_dict[key]) - int(len(paths_dict[key]) * test_data_proportion)
    
    copy_images(0, test_index, paths_dict[key], train_dir)
    copy_images(test_index, len(paths_dict[key]), paths_dict[key], test_dir)

<div dir="rtl">در اینجا یادگیری انتقالی با استفاده از inception_v3 انجام شده است. هر چند گزینه های متفاوتی برای این کار وجود دارد.</div>

In [ ]:
# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

<div dir="rtl">وزن ها نیز از شبکه imagenet گرفته میشود.</div>

In [ ]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

<div dir="rtl">مانند تمرین های پیش با قرار دادن این شرط بر روی هر epoch از overfit شدن شبکه جلوگیری میکنیم.</div>

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.959):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

کراس به صورت پیش فرض این معیارهای سنجش را ندارد. پس ما آنها را اضافه میکنیم.

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives =K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


<div dir="rtl">در ادامه مدل را با چند گزینه متفاوت تست میکنیم و نتایج را ترسیم میکنیم.</div>

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(pre_trained_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)           
model = Model( pre_trained_model.input, x) 
model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc', f1_m, precision_m, recall_m])

In [ ]:

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))
validation_generator =  test_datagen.flow_from_directory( val_data_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

In [ ]:
callbacks = myCallback()
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 100,
            validation_steps = 50,
            verbose = 2,
            callbacks=[callbacks])

In [ ]:
print("F1-score: ",history.history['f1_m'])
print("F1-score val: ",history.history['val_f1_m'])
print("Precision: ",history.history['precision_m'])
print("Precision val: ",history.history['val_precision_m'])
print("Recall: ",history.history['recall_m'])
print("Recall: ",history.history['val_recall_m'])

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7') #Only use layers including and above 'mixed7'
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc', f1_m, precision_m, recall_m])

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

validation_generator =  test_datagen.flow_from_directory( val_data_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

callbacks = myCallback()
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 100,
            validation_steps = 50,
            verbose = 2,
            callbacks=[callbacks])

In [ ]:
print("F1-score: ",history.history['f1_m'])
print("F1-score val: ",history.history['val_f1_m'])
print("Precision: ",history.history['precision_m'])
print("Precision val: ",history.history['val_precision_m'])
print("Recall: ",history.history['recall_m'])
print("Recall: ",history.history['val_recall_m'])

In [ ]:
len_train_data = len(train_generator.filenames)
len_test_data = len(validation_generator.filenames)

result = model.evaluate_generator(validation_generator,
                                  len_test_data // 20,
                                  verbose=1)

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Train accuracy')
plt.plot(epochs, val_acc, 'b', label='Val accuracy')
plt.title('Training & validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Train Loss')
plt.plot(epochs, val_loss, 'b', label='Val Loss')
plt.title('Training & validation loss')
plt.legend()

plt.show()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7') #Only use layers including and above 'mixed7'
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc', f1_m, precision_m, recall_m])

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

validation_generator =  test_datagen.flow_from_directory( val_data_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

callbacks = myCallback()
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 100,
            validation_steps = 50,
            verbose = 2,
            callbacks=[callbacks])

In [ ]:
print("F1-score: ",history.history['f1_m'])
print("F1-score val: ",history.history['val_f1_m'])
print("Precision: ",history.history['precision_m'])
print("Precision val: ",history.history['val_precision_m'])
print("Recall: ",history.history['recall_m'])
print("Recall: ",history.history['val_recall_m'])

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#batch_size = 20

result = model.evaluate_generator(validation_generator,
                                  len_test_data // 20,
                                  verbose=1)

In [ ]:
model.summary()

In [ ]:
model.evaluate(test_gen)